In [91]:

import os
from os.path import join, dirname
from os import getcwd
from PIL import Image, ImageDraw
import pandas as pd

PATH_REPO = dirname(getcwd())
PATH_DATA = join(PATH_REPO, "data")
PATH_FOLDER = join(PATH_DATA, "20241106_yolo")
PATH_FOLDER

train=pd.read_csv(join(PATH_FOLDER,'labels','train', "20241106_yolo_csv.csv"))
val=pd.read_csv(join(PATH_FOLDER, 'labels','val', "20241106_yolo_csv.csv"))
val

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,"FRANOM22_COLH78_0487_0293.jpg,839638,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
1,"FRANOM22_COLH78_0487_0293.jpg,839638,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
2,"FRANOM22_COLH78_0487_0293.jpg,839638,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
3,"FRANOM22_COLH78_0487_0293.jpg,839638,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
4,"FRANOM22_COLH78_0487_0293.jpg,839638,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
5,"FRANOM58_078MIOM0841_0028.jpg,966062,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
6,"FRANOM58_078MIOM0841_0028.jpg,966062,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
7,"FRANOM58_078MIOM0841_0028.jpg,966062,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
8,"FRANOM58_078MIOM0841_0028.jpg,966062,""{""""publi...",NaN,NaN,NaN,NaN,NaN,NaN
9,"FRANOM58_078MIOM1282_0434.jpg,1079224,""{""""publ...",NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
# Load exported csv and save into corresponding train and val folder
annotations=pd.read_csv(r"C:\Users\beigelman\Downloads\via_export_csv (2).csv",sep=";")
# List unique filenames
annotations
unique_filenames = annotations['filename'].unique()


In [187]:
#NOTE: lorsque export csv depuis VGG, les file_attributes ne sont pas vides et non consistants d'une image à l'autre. 
#solution: ouvrir le csv dans excel, séparer les colonnes par ",", supprimer la colonne file_attributes
#ici, importer le csv dans pandas, puis sauvegarder les annotations dans un nouveau csv pour train et val

# OPTION 1 : Calculate 90% of unique filenames
#num_train_filenames = int(len(unique_filenames) * 0.9)
#train_filenames = unique_filenames[:num_train_filenames]
#val_filenames = unique_filenames[num_train_filenames:]

# Filter annotations for train and val datasets
#train_annotations = annotations[(annotations['filename'].isin(train_filenames)) & (annotations['region_shape_attributes'] != '{}')]
#val_annotations = annotations[(annotations['filename'].isin(val_filenames)) & (annotations['region_shape_attributes'] != '{}')]


#OPTION 2: Select manually the validation images
val_filenames=['FRANOM22_COLH78_0487_0293.jpg','FRANOM58_078MIOM0841_0028.jpg','FRANOM58_078MIOM1282_0434.jpg']
val_annotations = annotations[(annotations['filename'].isin(val_filenames)) & (annotations['region_shape_attributes'] != '{}')]
train_annotations= annotations[~annotations['filename'].isin(val_filenames) & (annotations['region_shape_attributes'] != '{}')]


# Save train and val datasets to CSV
train_annotations.to_csv(join(PATH_FOLDER,'labels','train', "20241106_yolo_csv.csv"),index=True)
val_annotations.to_csv(join(PATH_FOLDER, 'labels','val', "20241106_yolo_csv.csv"),index=True)


In [191]:
val_filenames, val_annotations

(['FRANOM22_COLH78_0487_0293.jpg',
  'FRANOM58_078MIOM0841_0028.jpg',
  'FRANOM58_078MIOM1282_0434.jpg'],
                           filename  file_size  region_count  region_id  \
 105  FRANOM22_COLH78_0487_0293.jpg     839638             5          0   
 106  FRANOM22_COLH78_0487_0293.jpg     839638             5          1   
 107  FRANOM22_COLH78_0487_0293.jpg     839638             5          2   
 108  FRANOM22_COLH78_0487_0293.jpg     839638             5          3   
 109  FRANOM22_COLH78_0487_0293.jpg     839638             5          4   
 110  FRANOM58_078MIOM0841_0028.jpg     966062             4          0   
 111  FRANOM58_078MIOM0841_0028.jpg     966062             4          1   
 112  FRANOM58_078MIOM0841_0028.jpg     966062             4          2   
 113  FRANOM58_078MIOM0841_0028.jpg     966062             4          3   
 114  FRANOM58_078MIOM1282_0434.jpg    1079224             3          0   
 115  FRANOM58_078MIOM1282_0434.jpg    1079224             3         

In [192]:
def process_image_set(df, path_folder, subset='train'):
    print(f"Processing {subset} set...")
    source_images_dir = join(path_folder, "images", subset)
    output_dir = join(path_folder, "labels", subset)

    unique_images = df['filename'].unique()
    print(f"Unique images in {subset} set: {len(unique_images)}")

    for img_name in unique_images:
        img_annotations = df[df['filename'] == img_name]
        print(f"Processing image: {img_name}")
        
        img = Image.open(join(source_images_dir, img_name))
        img_width = img.width
        img_height = img.height
        
        label_lines = []
        for _, row in img_annotations.iterrows():
            # Extract the coordinates
            region_attrs = eval(row['region_shape_attributes'])
            x = region_attrs['x']
            y = region_attrs['y']
            w = region_attrs['width']
            h = region_attrs['height']
            
            # Convert in YOLO  format (normalized)
            x_center = (x + w/2) / img_width
            y_center = (y + h/2) / img_height
            width = w / img_width
            height = h / img_height
            
            # Get the class
            class_name = eval(row['region_attributes'])['Class']
            class_id = class_mapping[class_name]
            
            # Créer la ligne au format YOLO
            label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        
        # Write the label file
        label_path = join(output_dir, os.path.splitext(img_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            f.write('\n'.join(label_lines))
        print(f"Label file written for {img_name}")
    print(f"Processing of {subset} set completed.")


In [196]:
# Process the train and val sets
process_image_set(train_annotations, PATH_FOLDER,'train')
print('.........')
process_image_set(val_annotations, PATH_FOLDER,'val')

Processing train set...
Unique images in train set: 41
Processing image: FRANOM22_COLH78_0261_0232.jpg
Label file written for FRANOM22_COLH78_0261_0232.jpg
Processing image: FRANOM22_COLH78_0261_0233.jpg
Label file written for FRANOM22_COLH78_0261_0233.jpg
Processing image: FRANOM22_COLH78_0261_0234.jpg
Label file written for FRANOM22_COLH78_0261_0234.jpg
Processing image: FRANOM22_COLH78_0261_0235.jpg
Label file written for FRANOM22_COLH78_0261_0235.jpg
Processing image: FRANOM22_COLH78_0261_0236.jpg
Label file written for FRANOM22_COLH78_0261_0236.jpg
Processing image: FRANOM58_078MIOM0841_0008.jpg
Label file written for FRANOM58_078MIOM0841_0008.jpg
Processing image: FRANOM58_078MIOM0841_0009.jpg
Label file written for FRANOM58_078MIOM0841_0009.jpg
Processing image: FRANOM58_078MIOM1282_0414.jpg
Label file written for FRANOM58_078MIOM1282_0414.jpg
Processing image: FRANOM22_COLH78_0484_0220.jpg
Label file written for FRANOM22_COLH78_0484_0220.jpg
Processing image: FRANOM22_COLH78_04